<a href="https://colab.research.google.com/github/Kritarth-Dandapat/CSE-473-Marine-Guardian/blob/master/Marine_Guardian_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import os
from pathlib import Path
import glob
from scipy.fft import fft2, fftshift
from scipy import ndimage
import random

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kritarthdandapat2006","key":"3ce3ab5ee491850819638274f0b18087"}'}

In [6]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

In [7]:
import os
import subprocess

dataset_slug = "rhammell/ships-in-satellite-imagery"
destination_path = "./ships_dataset"

# Create destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# Path to the zip file
zip_file_path = os.path.join(destination_path, f"{dataset_slug.split('/')[1]}.zip")

# Check if the dataset already exists
if os.path.exists(zip_file_path):
    print(f"Dataset '{dataset_slug}' already exists at '{destination_path}'. Skipping download.")
else:
    try:
        # Download the dataset if not already present
        command = ["kaggle", "datasets", "download", "-d", dataset_slug, "-p", destination_path]
        subprocess.run(command, check=True)
        print(f"Dataset '{dataset_slug}' downloaded successfully to '{destination_path}'.")
    except subprocess.CalledProcessError as e:
        print(f"Error downloading dataset '{dataset_slug}': {e}")
    except FileNotFoundError:
        print("Kaggle API not found. Please ensure the Kaggle CLI is installed and configured.")
        print("You can install it using: pip install kaggle")
        print("And configure it by placing your kaggle.json file in ~/.kaggle/")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

Dataset 'rhammell/ships-in-satellite-imagery' downloaded successfully to './ships_dataset'.


In [8]:
!unzip -q /content/ships_dataset/ships-in-satellite-imagery.zip -d /content/ships_dataset/

In [2]:

def load_grayscale_images(dataset_dir, img_size=(256, 256), max_samples=None, seed=None):
    """
    Load grayscale images from a single directory, resize them,
    and optionally limit the number of samples.
    """
    rng = np.random.default_rng(seed) if seed is not None else np.random.default_rng()
    image_paths = glob.glob(os.path.join(dataset_dir, '*.jpg')) + glob.glob(os.path.join(dataset_dir, '*.png'))

    if max_samples is not None and len(image_paths) > max_samples:
        rng.shuffle(image_paths)
        image_paths = image_paths[:max_samples]

    images = []
    for img_path in image_paths:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)

    return np.array(images)

In [3]:
def calculate_roundness(gray_images):
    all_roundness_values = []

    for img in images:
        # 1. Convert to Grayscale
        if len(img.shape) > 2 and img.shape[2] > 1:
            gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Or COLOR_RGB2GRAY
        else:
            gray_image = img.copy()

        # 2. Thresholding (if necessary)
        if gray_image.dtype != np.uint8:
            gray_image = (gray_image * 255).astype(np.uint8)
        _, thresh = cv2.threshold(gray_image, 127, 255, cv2.THRESH_BINARY)

        # 3. Find Contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 4. Handle No Contours
        if not contours:
            all_roundness_values.append(None)
            continue

        # 5. Find the Largest Contour
        largest_contour = max(contours, key=cv2.contourArea)

        # 6. Calculate Area and Perimeter
        area = cv2.contourArea(largest_contour)
        perimeter = cv2.arcLength(largest_contour, True)

        # 7. Handle Zero Perimeter
        if perimeter == 0:
            all_roundness_values.append(0.0)
            continue

        # 8. Calculate Roundness
        roundness = (4 * np.pi * area) / (perimeter * perimeter)
        all_roundness_values.append(roundness)

    return np.array(all_roundness_values)

In [4]:
def filter_images_by_roundness(images, roundness_values, min_val=0.15, max_val=0.5):
    """
    Returns images and roundness values where roundness is between min_val and max_val (inclusive).

    Args:
        images (list): List of images (numpy arrays).
        roundness_values (list): Corresponding roundness values.
        min_val (float): Minimum roundness value.
        max_val (float): Maximum roundness value.

    Returns:
        filtered_images (list): Images with roundness in the specified range.
        filtered_roundness (list): Corresponding roundness values.
    """
    filtered_images = []
    filtered_roundness = []

    for img, r in zip(images, roundness_values):
        if r is not None and min_val <= r <= max_val:
            filtered_images.append(img)
            filtered_roundness.append(r)

    return filtered_images, filtered_roundness

In [14]:
images = load_grayscale_images("/content/ships_dataset/shipsnet/shipsnet", img_size=(128, 128), max_samples=4000, seed=42)

In [15]:
roundness_values = calculate_roundness(images)

In [16]:
filtered_imgs, filtered_roundness = filter_images_by_roundness(images, roundness_values)
print(f"Found {len(filtered_imgs)} images with roundness between 0.15 and 0.50.")

Found 2402 images with roundness between 0.15 and 0.50.
